# TP 2 - Exercício 1

Neste exercicio nos foi proposto o seguinte:


1. Construir uma classe Python que implemente o  EdDSA a partir do “standard” FIPS186-5
    1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
    2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.
    
Primeiramente começamos por importar as bibliotecas do SageMath, Hashlib e Random conforme abaixo.

In [1]:
from sage import *
import hashlib
import random

Para realizar os objetivos propostos, contruímos a classe abaixo, onde nela definimos os parametros para as curvas Edwards 25519 e 448. Esta classe recebe o nome da curva solicitada para definir os parametros a serem usados posteriormente para assinatura e verificação da assinatura.

###### Curva Edwards25519:
Esta curva, usada para segurança de 128 bits, tem por parametros o prime $ p = 2^{255} -19 $, d para essa curva é calculado através da formula $ d = -\frac{(121665)}{(121666)} * mod(p) $ o que poderia ter sido setado como constante, visto que está presente na norma RFC 7748.

Outros parâmetros importantes desta curva são os *Base Points*, onde Gx foi definido como na norma e Gy calculado através da formula.
    
###### Curva Edwards448: 
Para esta curva, usada em segurança de 224 bits, tem por parametros o prime calculado através da formula $ p = 2^{448} - 2^{224} - 1 $, d e os pontos base Gx e Gy foram definidos conforme a RFC 7748.
    
    
Quandos os pontos base são definidos eles passam por uma verificação de consistencia, através da formula abaixo, para identificar se pertencem realmente a curva designada.
    $$
    ((a \cdot Gx^2 + Gy^2) \mod p) = ((1 \cdot d \cdot Gx^2 \cdot Gy^2) \mod p + 1)
    $$
    
Onde ambos os lados da equação devem ser iguais para continuar a execução do código.
    
Nesta classe ainda são definidos os métodos de *add_point*, que recebe dois pontos (P, Q), usado para somar pontos através das formulas de Edwards. E, por fim, o método *scalar_multiplication*, que recebe um inteiro e um ponto (k, Q), utilizado para multiplicar um ponto na curva por um valor inteiro, utiliza da técnica de dobrar e somar para realizar essa multiplicação, pois este algoritmo aproveita a representação binária do inteiro **k** para realizar a multiplicação escalar de forma eficiente. Em cada passo, ele condicionalmente "adiciona" baseado no bit atual de **k** e "duplica" para se mover para a próxima posição binária. Esse método reduz significativamente o número de operações necessárias em comparação com a adição do ponto **Q** repetida **k** vezes.

In [2]:
class TwistedEdwardsCurve:
    def __init__(self, curve_name):
        if curve_name == "edwards25519":
            self.p = pow(2, 255) - 19
            self.a = -1
            self.d = (-121665 * inverse_mod(121666, self.p)) % self.p
            #self.d = 37095705934669439343138083508754565189542113879843219016388785533085940283555
            
            
        elif curve_name == "edwards448":
            self.p = (pow(2, 448) - pow(2, 224)) - 1
            self.a = 1
            self.d = 611975850744529176160423220965553317543219696871016626328968936415087860042636474891785599283666020414768678979989378147065462815545017
        else:
            raise ValueError("Curva inválida: " + curve_name)
    
    
    def BasePoint(self,curve_name):
        if curve_name == "edwards25519":
            u = 9
            Gy = ((u-1) * inverse_mod(u+1, self.p)) % self.p
            Gx = 15112221349535400772501151409588531511454012693041857206046113283949847762202
        
        elif curve_name == "edwards448":
            Gx = 345397493039729516374008604150537410266655260075183290216406970281645695073672344430481787759340633221708391583424041788924124567700732
            Gy = 363419362147803445274661903944002267176820680343659030140745099590306164083365386343198191849338272965044442230921818680526749009182718
        

        assert ((self.a * Gx*Gx + Gy*Gy) % self.p) == ((1 * self.d*Gx*Gx*Gy*Gy) %self.p + 1)
        return (Gx,Gy)
    
    def scalar_multiplication(self,k, Q):

        addition_point = Q
        
        k_binary = bin(k)[2:]
        
        for i in range(1, len(k_binary)):
            current_bit = k_binary[i:i+1]
            
            addition_point = self.add_point(addition_point, addition_point)
            if current_bit == '1':
                addition_point = self.add_point(addition_point, Q)
        return addition_point

    def add_point(self, P, Q):
        x1, y1 = P
        x2, y2 = Q
        
        x3 = (((x1 * y2 + y1 * x2) % self.p) * pow(1 + self.d*x1*x2*y1*y2, -1, self.p)) % self.p
        y3 = (((y1 * y2 - self.a*x1*x2) % self.p) * pow(1 - self.d*x1*x2*y1*y2, -1, self.p)) % self.p
        
        #print((self.a * x3*x3 +y3*y3) % self.p), print ((1 + self.d *x3*x3*y3*y3) % self.p)

        return (x3, y3)

###### Classe EdDSA

Nesta classe são iniciados os seguintes métodos:

*Hashing:* neste método recebe-se a mensagem já convertida para inteiro, converte novamente para string, calcula o Hash SHA-256 (utliziando a biblioteca hashlib) e então converte novamente para string.

*HashToInt:* este método recebe uma mensagem em string, converte em bytes utilizando o *"utf-8"* e por fim converte em inteiro.

*generate_keypair:* Este método gera a chave privada de 32 bytes (256 bits), chama o basepoint da curva selecionada, e então calcula a multiplicação escalar do basepoint com a chave privada gerada aleatoriamente para gerar a chave pública que é um ponto na curva selecionada.

*sign:* Método para a assinatura da mensagem, primeiro retorna-se os pontos base da curva selecionada, após isso é gerado um **r** a partir de um hashing duplo e modular da mensagem. Então com esses valores gera-se **R**, que tem como entradas o hashing **r** e o ponto base **G**. Um valor **h** é calculado como a soma modular de componentes que incluem a coordenada x de **R**, a coordenada x da **chave pública**, e a própria **mensagem**. A assinatura final **sig** é derivada somando-se **r** ao produto de **h** pela **chave privada**. 

*verify:* É a verificação da assinatura digital gerada em *sign*, este método checa a validade de uma assinatura *(R, sig)* para a mensagem, utilizando a chave pública **pk** do signatário. Ele recalcula o valor **h** a partir da mensagem, da chave pública, e do ponto R gerado durante a assinatura, e compara dois pontos na curva: **P1**, derivado da multiplicação escalar da assinatura **sig** pelo **ponto base**, e **P2**, obtido adicionando **R** ao resultado da multiplicação escalar de **h** pela **chave pública**. Se P1 e P2 forem iguais, a assinatura é considerada válida, indicando que foi criada com a chave privada correspondente à pk fornecida, autenticando assim a mensagem e o signatário de forma segura e eficaz.

In [3]:
class EdDSA:
    def __init__(self, curve_name):
        self.curve = TwistedEdwardsCurve(curve_name)
        
    def Hashing(self, m_int):
        return int(hashlib.sha256(str(m_int).encode('utf-8')).hexdigest(),16)
    
    def HashToInt(self, m):
        encoded_text = m.encode('utf-8')
        hex_text = encoded_text.hex()
        return int(hex_text, 16)

    def generate_keypair(self):
        sk = random.getrandbits(256)  # Chave privada: 32 bytes aleatórios
        basepoint = self.curve.BasePoint(curve_name)
        pk = self.curve.scalar_multiplication(sk,basepoint)
        return sk, pk

    def sign(self, sk, pk, message):
        
        G = self.curve.BasePoint(curve_name)
        
        r = self.Hashing(self.Hashing(message) + message) % self.curve.p
        
        R = self.curve.scalar_multiplication(r, G)
        
        h = (R[0] + pk[0] + message) % self.curve.p
        sig = (r + h * sk)
        
        return R, sig

        
    def verify(self, pk, R, sig, message):
        
        h = (R[0] + pk[0] + message) % self.curve.p
        
        basepoint = self.curve.BasePoint(curve_name)
        
        P1 = self.curve.scalar_multiplication(sig, basepoint)
        P2 = self.curve.add_point(R, self.curve.scalar_multiplication(h, pk))
        
        return P1 == P2

###### Assinatura com Edwards25519

In [4]:
curve_name = "edwards25519"

# Gerando par de chaves
sk, pk = EdDSA(curve_name).generate_keypair()
print("Chave Privada: ", sk)
print("Chave Pública: ", pk)

# Mensagem a ser assinada
message = "Hello EdDSA!"
message = EdDSA(curve_name).HashToInt(message)

#Assinatura
R, S = EdDSA(curve_name).sign(sk, pk, message)  
print("Assinatura: ", R, S)

#Verificação
verify_signature = EdDSA(curve_name).verify(pk, R, S, message)
print("Verificação?", verify_signature)

Chave Privada:  102259057467395372173978795110483991253376708401413165054480674281226561874814
Chave Pública:  (45710851312088909703149909446630110405275141585001759362850230992014056350425, 26281167175670247846113882961274529671478172874372998931517239316998308685400)
Assinatura:  (12399122663955051457979918901410424170681720296829101103839845937406703397287, 7997106330477378755358977030532607155405239964962537368257268094674858511848) 21876214450883973075024817951963381283330564710012600642557433358415466979198061710045491411018502564412121888039386631993503207825511120846098594836296
Verificação? True


###### Assinatura com Edwards448

In [5]:
curve_name = "edwards448"

# Gerando par de chaves
sk, pk = EdDSA(curve_name).generate_keypair()
print("Chave Privada: ", sk)
print("Chave Pública: ", pk)


# Mensagem a ser assinada
message = "Hello EdDSA!"
message = EdDSA(curve_name).HashToInt(message)

#Assinatura
R, S = EdDSA(curve_name).sign(sk, pk, message)
print("Assinatura: ", R, S)

#Verificação
verify_signature = EdDSA(curve_name).verify(pk, R, S, message)
print("Verificação?", verify_signature)

Chave Privada:  95487061131839445902126964064451372853631899609050493310411147687754337729923
Chave Pública:  (125875207009252637047608946324638036178681052128265271803384968888766265047390994534888431938105417269715644874151578851344134036741330, 638765934492699891121286728440873158678873444023324167012725070910491821887860198725971324501896019599260973490811967292817264460971312)
Assinatura:  (357292502741317297606648517611683476641698744177769878254907419623208300781806815498980241710813233638323655477233028645145213522279208, 131517953651434496917308253346310708103862294909294595867722208679753288572036249154800081691900237683363652424214224148730691609811329) 46136264637883529266411944118352077974057086834605026582899664711124199553790444464940977507505687043598255518835509454735948899499377995076238626053211104776550464204010823971108058461351203121318257146733722753
Verificação? True
